In [295]:
import pandas as pd
import numpy as np
import requests
from nba_api.stats.endpoints import playergamelog

In [296]:
curry = pd.DataFrame()
player_stats = playergamelog.PlayerGameLog(player_id='201939')
curry2022 = player_stats.player_game_log.get_data_frame()
seasons = ['2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']

In [297]:
for x in seasons:
    #pull player stats into dataframe
    player_stats = playergamelog.PlayerGameLog(player_id='201939', season=x, season_type_all_star='Regular Season')    
    df = player_stats.player_game_log.get_data_frame()
    
    #append to curry if dataframe has enough sample size (looking for >(82*45% = 36.9) games played that season)
    if len(df.index) > 36.9:
        curry = pd.concat([curry, df], ignore_index=True)

In [298]:
drop_columns = ['SEASON_ID','Player_ID','Game_ID','GAME_DATE','WL','MATCHUP','VIDEO_AVAILABLE',
                'FG_PCT','FG3_PCT','FT_PCT','REB','MIN']
curry = curry.drop(drop_columns, axis=1)
curry2022 = curry2022.drop(drop_columns, axis=1)

In [299]:
curry2022.loc['mean'] = curry2022.head(20).mean()
avg = curry2022.tail(1)

In [300]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

y_pred = pd.DataFrame()
pred_columns = ['PTS','AST','OREB', 'DREB']
for x in pred_columns:
    X = curry.loc[:, curry.columns != x]
    y = curry[x]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    print("Accuracy on Test: ", lr.score(X_test, y_test))
    
    y_error = lr.predict(X_test)
    print("MAE: ", mean_absolute_error(y_true=y_test, y_pred=y_error))
    
    y_pred[x] = lr.predict(avg.loc[:, avg.columns != x])

Accuracy on Test:  1.0
MAE:  7.204903340607416e-15
Accuracy on Test:  0.07077479861498104
MAE:  2.192429676281503
Accuracy on Test:  0.006430665766554822
MAE:  0.6770687167756316
Accuracy on Test:  0.07639073041547884
MAE:  1.709087063826816


In [301]:
y_pred

,PTS,AST,OREB,DREB
0,27.4,5.936149,0.585113,4.303346


# Since accuracy on stats is very low, WIP on finding features that will help increase accuracy